In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/KaiserHealthNews.txt'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,586260160462589954,Thu Apr 09 20:11:29 +0000 2015,A new sort of extracurricular activity: seeing...
1,586238751334125569,Thu Apr 09 18:46:24 +0000 2015,Houston firefighters have another tool at thei...
2,586181878635298817,Thu Apr 09 15:00:25 +0000 2015,Will #Montana expand its #Medicaid program? So...
3,586159308745920512,Thu Apr 09 13:30:44 +0000 2015,Rand Paul's campaign first day: abortion and t...
4,586159071264432128,Thu Apr 09 13:29:47 +0000 2015,"Enroll America, A Group Dedicated To Promoting..."


In [3]:
# -------------------------------------------------
frame.shape

(3508, 3)

In [4]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'A new sort of extracurricular activity: seeing patients. http://khne.ws/1ceRuaZ http://pbs.twimg.com/media/CCLQgiFUIAAC7nJ.jpg'

In [5]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(lambda row: split_txt_form_url(regex_url, row))
frame.head()

http://khne.ws/1ceRuaZ
http://khne.ws/1NfQjsp
http://khne.ws/1Do7UrB
http://khne.ws/1y8aJwC
http://khne.ws/1HUROpc
http://khne.ws/1JowAkr
http://khne.ws/1NaiTex
https://youtu.be/RSkNL32-3iE
http://khne.ws/1JowAkr
http://cnn.it/1yVmb9H
http://wamu.fm/1NQYhmG
http://kaiserhealthnews.org/news/khn-video-tax-deadline-meets-the-health-law/
http://kaiserhealthnews.org/news/rule-proposed-on-providing-mental-health-parity-in-medicaid-program/
http://kaiserhealthnews.org/news/battle-for-mental-health-parity-produces-mixed-results/
http://kaiserhealthnews.org/news/high-deductible-health-plans-can-ruin-finances/
http://www.npr.org/blogs/health/2015/04/06/397100748/maybe-you-should-skip-that-annual-physical
http://kaiserhealthnews.org/morning-breakout/billionaires-harness-money-technology-in-pursuit-of-fountain-of-youth/
http://kaiserhealthnews.org/news/ritual-not-science-keeps-the-annual-physical-alive/
http://kaiserhealthnews.org/news/medicare-is-stingy-in-first-year-of-doctor-bonuses/
http://kai

http://khne.ws/16TJVyX
http://khne.ws/HKMauB
http://khne.ws/17m17hj
http://khne.ws/15AFERb
http://khne.ws/1855prM
http://khne.ws/1dntMa0
http://khne.ws/12LpWll
http://khne.ws/1dPOE8D
http://khne.ws/19ZjqNb
http://khne.ws/19ZjqNb
http://khne.ws/1dPDH72
http://khne.ws/184mguM
http://khne.ws/17iaEWV
http://khne.ws/18iQOrv
http://khne.ws/146bB0l
http://khne.ws/18ihV68
http://khne.ws/17jGEIm
http://khne.ws/HKMauB
http://khne.ws/1fgqtOF
http://khne.ws/16FK2L8
http://khne.ws/178rNE1
http://khne.ws/1djILCR
http://4sq.com/177CdUw
http://khne.ws/1fedayb
http://n.pr/1777RSa
http://khne.ws/14VgYiZ
http://khne.ws/14VgYiZ
http://khne.ws/HKMauB
http://khne.ws/14V7TGS
http://khne.ws/1d9GJFo
http://khne.ws/13WQm0M
http://khne.ws/188DfLg
http://khne.ws/HKMauB
http://khne.ws/1f77xSv
http://khne.ws/SKbqCb
http://t.co/PH
http://khne.ws/12t4a5I
http://khne.ws/13J2QOO
http://www.kaiserhealthnews.org/Stories/2013/August/21/Insurance-For-Working-Spouses-At-UPS.aspx
http://khne.ws/HKMauB
http://khne.ws/13FSX4r


http://khne.ws/1Bobykn
http://khne.ws/1E3vyWK
http://khne.ws/1Bszf9R
http://khne.ws/1Bszf9R
http://khne.ws/1FB35Kd
http://khne.ws/1KLwiZi
http://khne.ws/1McB2mr
http://khne.ws/1Flltq9
http://khne.ws/1CI4JMb
http://khne.ws/1A0DnLq
http://khne.ws/1CBy3E2
http://khne.ws/1DYN5le
http://khne.ws/1AKPo6I
http://khne.ws/1BSVEjP
http://khne.ws/1DNNogc
http://khne.ws/1BSSpJh
http://khne.ws/18PvMJf
http://to.pbs.org/1vYH47Y
http://khne.ws/1GfTiwn
http://khne.ws/1zEJu6V
http://khne.ws/1zVa24W
http://khne.ws/1Kdwu3h
http://kaiserhealthnews.org/news/5-things-to-know-about-the-supreme-court-case-challenging-the-health-law/
http://khne.ws/1M22Hs1
http://khne.ws/1ALjtqa
http://kaiserhealthnews.org/news/when-health-care-is-far-from-home/
http://kaiserhealthnews.org/news/the-extra-cost-of-extra-weight-for-older-adults/
http://kaiserhealthnews.org/news/more-than-one-in-four-foster-kids-miss-required-checkups/
http://kaiserhealthnews.org/news/internists-get-a-break-from-controversial-efforts-to-bolster-per

http://khne.ws/17Lw3e1
http://khne.ws/17Lw3e1
http://khne.ws/17m4tlf
http://khne.ws/IeQJim
http://khne.ws/17lFb6F
http://khne.ws/1eghIG1
http://khne.ws/1deGKYS
http://khne.ws/HKMauB
http://khne.ws/1dbQcMT
http://khne.ws/17GyU85
http://khne.ws/1bJLCC9
http://khne.ws/1hScPaW
http://khne.ws/1byAjJs
http://www.sacbee.com/2013/11/15/5915604/california-sends-misinformation.html
http://khne.ws/19qwpkr
http://khne.ws/1bsWMHN
http://khne.ws/177XtIB
http://khne.ws/1j9zzyw
http://khne.ws/1hLrWD2
http://khne.ws/17wQonl
http://khne.ws/17wQonl
http://khne.ws/17wQonl
http://khne.ws/19nLMds
http://khne.ws/17VQxRP
http://khne.ws/17s6w4c
http://khne.ws/1bxyJei
http://khne.ws/1cshVns
http://khne.ws/17s6w4c
http://khne.ws/1e423JD
http://khne.ws/18sFFoS
http://Healthcare.gov
http://khne.ws/1j4PCO5
http://healthcare.gov
http://khne.ws/1bs5iu9
http://khne.ws/1hD7N1Y
http://khne.ws/1anaRJf
http://khne.ws/18oBla2
http://bit.ly/17qgBDX
http://www.kaiserhealthnews.org/Stories/2013/November/09/Massachussets-xray-

http://khne.ws/TwECA3
http://www.kaiserhealthnews.org/Multimedia/2012/December/2013-reporter-predictions-Gold.aspx
http://khne.ws/HKMauB
http://khne.ws/WjaPbu
http://khne.ws/WjaPbu
http://khne.ws/VneYff
http://ow.ly/1QCrqz
http://ow.ly/1QCrqz
http://khne.ws/YODB9X
http://khne.ws/VaNewS
http://khne.ws/Tc6Jo2
http://khne.ws/WuOytc
http://khne.ws/valuebased
http://n.pr/12CGDNQ
http://khne.ws/Tc6Jo2
http://khne.ws/HKMauB
http://khne.ws/12EiE0D
http://khne.ws/valuebased
http://www.kaiserhealthnews.org/Stories/2012/December/21/medicare-hospitals-value-based-purchasing.aspx
http://khne.ws/V91A0w
http://khne.ws/R7lmcp
http://khne.ws/U0IBCd
http://khne.ws/HKMauB
http://khne.ws/MY6CKf
http://khne.ws/ZJMpOR
http://khne.ws/Wsgfqn
http://khne.ws/Tzjac3
http://khne.ws/12pnPCA
http://khne.ws/R54xyZ
http://khne.ws/HKMauB
http://khne.ws/RCptPp
http://khne.ws/SNtalw
http://khne.ws/SNfy9H
http://khne.ws/ZJMpOR
http://khne.ws/UOEOJ5
http://khne.ws/UOEOJ5
http://khne.ws/HKMauB
http://khne.ws/T2OTCs
http://

,tweetid,date,title,url
0,586260160462589954,Thu Apr 09 20:11:29 +0000 2015,A new sort of extracurricular activity: seeing...,http://khne.ws/1ceRuaZ
1,586238751334125569,Thu Apr 09 18:46:24 +0000 2015,Houston firefighters have another tool at thei...,http://khne.ws/1NfQjsp
2,586181878635298817,Thu Apr 09 15:00:25 +0000 2015,Will #Montana expand its #Medicaid program? So...,http://khne.ws/1Do7UrB
3,586159308745920512,Thu Apr 09 13:30:44 +0000 2015,Rand Paul's campaign first day: abortion and t...,http://khne.ws/1y8aJwC
4,586159071264432128,Thu Apr 09 13:29:47 +0000 2015,"Enroll America, A Group Dedicated To Promoting...",http://khne.ws/1HUROpc


In [6]:
# -------------------------------------------------
frame_urls = frame['url'].values

In [7]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key,url in enumerate(frame_urls):

    #     url = 'http://bit.ly/VymaIc'
    try:
        # Start the connection
        print(url)
        if len(url):
            if "http://" not in url:
                url = "http://" + url
            res = requests.get(url)

            # Check status header
            if res.status_code == 200:
                print("Header: ", 200)

                # Initialize BeautifulSoup()
                soup = BeautifulSoup(res.content, 'lxml')

                # Get return content as text
                # text_html = res.text

                # Add columns to dataframe
                if title_attempt:
                    df_temp = pd.DataFrame(columns=['author','content','tags'])
                    frame = pd.concat([frame,df_temp])
                    title_attempt = False

                ahref = soup.find('a',{'class':'article_link'})
                if ahref:
                    author = ahref.text            
        #             frame['author'][key] = author[:author.index(",")].lstrip()
                    frame['author'][key] = author.lstrip()
                else:
                    frame['author'] = ''
                frame['content'][key] = soup.find('div',{'class':'article-body'})
            else:
                print("[ERROR]: ", res.status_code)
        else:
            frame['author'][key] = ''
            frame['content'] = ''
        frame['tags'][key] = get_hastags_out(regex_hastags,frame['title'][key])
        frame['title'][key] = stript_hastags(frame['title'][key])
    except (RuntimeError, TypeError, NameError):
        pass

http://khne.ws/1ceRuaZ
[ERROR]:  404


KeyError: 'tags'

In [ ]:
# -------------------------------------------------
frame.head()

In [ ]:
# -------------------------------------------------
frame.shape